<a href="https://colab.research.google.com/github/varunbhanot/NLP/blob/master/BERT_Text_Classification_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import os
print(tf.__version__)

2.2.0


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 5.2MB/s 
     |████████████████████████████████| 3.0MB 13.1MB/s 
     |████████████████████████████████| 890kB 55.8MB/s 
     |████████████████████████████████| 1.1MB 56.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bd033f5b9bf1caffc066a26cc79ef99ffb3210094e6bf6382b95172d8aefac50
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"varunbhanot","key":"a50176b4894d0c7456bce55e6641e60d"}'}

In [4]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 35% 9.00M/25.7M [00:00<00:00, 19.8MB/s]
100% 25.7M/25.7M [00:00<00:00, 40.6MB/s]


In [7]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [8]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.model_selection import train_test_split

from transformers import (TFBertForSequenceClassification, 
                          BertTokenizer)

from tqdm import tqdm

In [9]:
data = pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
label_encoder = preprocessing.LabelEncoder()
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
X = (np.array(data['review']))
y = (np.array(data['sentiment']))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
print("Train dataset shape: {0}, \nTest dataset shape: {1}".format(X_train.shape, X_test.shape))

Train dataset shape: (40000,), 
Test dataset shape: (10000,)


In [1]:
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

NameError: ignored

In [13]:
pad_token=0
pad_token_segment_id=0
max_length=128

def convert_to_input(reviews):
  input_ids,attention_masks,token_type_ids=[],[],[]
  
  for x in tqdm(reviews,position=0, leave=True):
    inputs = bert_tokenizer.encode_plus(x,add_special_tokens=True, max_length=max_length,truncation=True)
    
    i, t = inputs["input_ids"], inputs["token_type_ids"]
    m = [1] * len(i)

    padding_length = max_length - len(i)

    i = i + ([pad_token] * padding_length)
    m = m + ([0] * padding_length)
    t = t + ([pad_token_segment_id] * padding_length)
    
    input_ids.append(i)
    attention_masks.append(m)
    token_type_ids.append(t)
  
  return [np.asarray(input_ids), 
            np.asarray(attention_masks), 
            np.asarray(token_type_ids)]

In [14]:
X_test_input=convert_to_input(X_test)
X_train_input=convert_to_input(X_train)

100%|██████████| 40000/40000 [01:40<00:00, 396.61it/s]


In [15]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y
train_ds = tf.data.Dataset.from_tensor_slices((X_train_input[0],X_train_input[1],X_train_input[2],y_train)).map(example_to_features).shuffle(100).batch(32).repeat(5)


test_ds=tf.data.Dataset.from_tensor_slices((X_test_input[0],X_test_input[1],X_test_input[2],y_test)).map(example_to_features).batch(64)


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [17]:
print("Fine-tuning BERT on IMDB")
bert_history = bert_model.fit(train_ds, epochs=3, validation_data=test_ds)

Fine-tuning BERT on IMDB
Epoch 1/3
6250/6250 [==============================] - 3018s 483ms/step - loss: 0.1400 - accuracy: 0.9424 - val_loss: 0.5289 - val_accuracy: 0.8854
Epoch 2/3
 152/6250 [..............................] - ETA: 47:52 - loss: 0.0300 - accuracy: 0.9881

KeyboardInterrupt: ignored

In [ ]:
def example_to_features_predict(inputs):
  return {"input_ids": inputs[0],
          "attention_mask": inputs[1],
          "token_type_ids": inputs[2]}

In [ ]:
X_test[5:8],y_test[5:8]

(array(['On the pure theatrical side, Last Stand was great, as the reenactments and soundtrack are very entertaining, but there are better accounts of this battle found elsewhere that, while not as long or as flashy, are far more historically comprehensive.<br /><br />Certain little details, such as the misuse of the word "hoplon" for the Greek hoplite shield and the mispronounciations of various names and words, really ate at me.<br /><br />My guess would be, that because "Last Stand of the 300" was aired the eve of the theatrical release of "300", the History Channel was only trying to ride the coattails of the movie\'s hype.<br /><br />If you\'re looking for a depiction that\'s historically accurate in all respects possible, you\'d have better luck elsewhere.',
        "No. I'm not kidding with this one. He was a guest reviewer for Entertainment Weekly and gave this movie positive marks. And who can blame him? This is a charming, upbeat, and rather funny Disney movie. Who doesn't lo

In [ ]:
X_predict_input=convert_to_input(X_test[1:4])

100%|██████████| 3/3 [00:00<00:00, 108.01it/s]


In [ ]:
predict_ds=tf.data.Dataset.from_tensor_slices(X_predict_input).map(example_to_features_predict).batch(2)

In [ ]:
bert_model.predict_on_batch(test_ds)

(<tf.Tensor: shape=(64, 2), dtype=float32, numpy=
 array([[ 0.68230027,  0.04853292],
        [ 1.5659124 , -1.2430305 ],
        [-0.30376363,  0.4779889 ],
        [ 1.5845535 , -0.9463671 ],
        [ 0.8431733 , -0.42819792],
        [ 0.4753108 ,  0.16980031],
        [-0.71908486,  0.7445457 ],
        [ 0.21682754,  0.24314192],
        [-1.0483649 ,  1.0591835 ],
        [ 0.8442112 , -0.12866004],
        [-0.690723  ,  0.741695  ],
        [-0.52187204,  0.5827493 ],
        [-0.73065066,  0.6999336 ],
        [ 1.5705034 , -1.3012695 ],
        [ 0.51423615, -0.05937077],
        [ 1.371553  , -1.1206478 ],
        [ 1.7374214 , -1.2659613 ],
        [-0.65970606,  0.69999844],
        [ 0.7414726 , -0.29334533],
        [ 1.7821999 , -1.2459989 ],
        [-0.9383836 ,  0.8871614 ],
        [-0.3621861 ,  0.38542902],
        [-1.5017791 ,  1.5198052 ],
        [-1.1472266 ,  1.1290525 ],
        [-0.78660816,  0.78397816],
        [ 1.1351368 , -0.35748467],
        [ 0.32